In [1]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
import itertools
import shapefile
import pandas as pd
import datetime
import requests
import zipfile
try:
    from StringIO import StringIO
except ImportError:
    from io import BytesIO as StringIO
import os
output_notebook()

Loading BokehJS ...

## U.S. Counties Map

In [2]:
state_fips = pd.read_csv('http://www2.census.gov/geo/docs/reference/state.txt', sep='|')
state_fips.drop('STATE', axis=1, inplace=True)
county_fips = pd.read_csv('http://www2.census.gov/geo/docs/reference/codes/files/national_county.txt', header=None)
county_fips.columns = ['STATE', 'STATEFP', 'COUNTYFP', 'COUNTYNAME', 'CLASSFP']
all_fips = pd.merge(county_fips, state_fips, left_on='STATE', right_on='STUSAB')

In [3]:
all_fips.head()

,STATE,STATEFP,COUNTYFP,COUNTYNAME,CLASSFP,STUSAB,STATE_NAME,STATENS
0,AL,1,1,Autauga County,H1,AL,Alabama,1779775
1,AL,1,3,Baldwin County,H1,AL,Alabama,1779775
2,AL,1,5,Barbour County,H1,AL,Alabama,1779775
3,AL,1,7,Bibb County,H1,AL,Alabama,1779775
4,AL,1,9,Blount County,H1,AL,Alabama,1779775


In [4]:
# https://www.census.gov/geo/maps-data/data/cbf/cbf_counties.html
shp = None
dbf = None
last_year = datetime.datetime.now().year - 1
url = 'http://www2.census.gov/geo/tiger/GENZ{}/shp/cb_{}_us_county_20m.zip'.format(last_year, last_year)  # Lowest
#url = 'http://www2.census.gov/geo/tiger/GENZ{}/shp/cb_{}_us_county_5m.zip'.format(last_year, last_year)
#url = 'http://www2.census.gov/geo/tiger/GENZ{}/shp/cb_{}_us_county_500k.zip'.format(last_year, last_year)  #Highest
response = requests.get(url)
with zipfile.ZipFile(StringIO(response.content)) as z:
    for fname in z.namelist():
        name, ext = os.path.splitext(fname)
        if ext == '.shp':
            shp = StringIO(z.read(fname)) 
            #shp = z.open(fname)
        elif ext == '.dbf':
            #dbf = z.open(fname)
            dbf = StringIO(z.read(fname))             
        else:
            pass
sf = shapefile.Reader(shp=shp, dbf=dbf)

In [5]:
lats = []
lons = []
county = []
state = []
for shprec in sf.shapeRecords():
    statefp = int(shprec.record[0])
    countyfp = int(shprec.record[1])
    fip = all_fips.loc[(all_fips['STATEFP'] == statefp) & (all_fips['COUNTYFP'] == countyfp)]
    county.append(fip['COUNTYNAME'])
    state.append(fip['STATE_NAME'])
    
    lat, lon = map(list, zip(*shprec.shape.points))
    lat = [l if l < 0 else l-360 for l in lat]
    indices = shprec.shape.parts.tolist()
    lat = [lat[i:j] + [float('NaN')] for i, j in zip(indices, indices[1:]+[None])]
    lon = [lon[i:j] + [float('NaN')] for i, j in zip(indices, indices[1:]+[None])]
    lat = list(itertools.chain.from_iterable(lat))
    lon = list(itertools.chain.from_iterable(lon))
    lats.append(lat)
    lons.append(lon)

df = pd.DataFrame({'x': lats, 'y': lons, 'county': county, 'state': state})
cds = ColumnDataSource(df)
p = figure(width=800)
county_patches = p.patches('x', 'y', source=cds, line_color='white')
hover = HoverTool(renderers=[county_patches], tooltips=[("County, State", "@county, @state")])
p.add_tools(hover)

In [6]:
show(p)

## U.S. States Map

In [7]:
# http://www.naturalearthdata.com/features/
shp = None
dbf = None
url = 'http://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_1_states_provinces.zip'
response = requests.get(url)
with zipfile.ZipFile(StringIO(response.content)) as z:
    for fname in z.namelist():
        name, ext = os.path.splitext(fname)
        if ext == '.shp':
            shp = StringIO(z.read(fname)) 
            #shp = z.open(fname)
        elif ext == '.dbf':
            #dbf = z.open(fname)
            dbf = StringIO(z.read(fname))             
        else:
            pass
sf = shapefile.Reader(shp=shp, dbf=dbf)

In [8]:
lats = []
lons = []
country = []
for shprec in sf.shapeRecords():
    name_long = unicode(shprec.record[12], 'cp1252').encode('utf8')
    country.append(name_long)
    lat, lon = map(list, zip(*shprec.shape.points))
    indices = shprec.shape.parts.tolist()
    lat = [lat[i:j] + [float('NaN')] for i, j in zip(indices, indices[1:]+[None])]
    lon = [lon[i:j] + [float('NaN')] for i, j in zip(indices, indices[1:]+[None])]
    lat = list(itertools.chain.from_iterable(lat))
    lon = list(itertools.chain.from_iterable(lon))
    lats.append(lat)
    lons.append(lon)

df = pd.DataFrame({'x': lats, 'y': lons, 'country': country})
cds = ColumnDataSource(df)
p = figure(width=800)
country_patches = p.patches('x', 'y', source=cds, line_color='white')
hover = HoverTool(renderers=[country_patches], tooltips=[('Country', '@country')])
p.add_tools(hover)

In [9]:
show(p)

## World Countries Map

In [10]:
# http://www.naturalearthdata.com/features/
shp = None
dbf = None
url = 'http://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip'
response = requests.get(url)
with zipfile.ZipFile(StringIO(response.content)) as z:
    for fname in z.namelist():
        name, ext = os.path.splitext(fname)
        if ext == '.shp':
            shp = StringIO(z.read(fname)) 
            #shp = z.open(fname)
        elif ext == '.dbf':
            #dbf = z.open(fname)
            dbf = StringIO(z.read(fname))             
        else:
            pass
sf = shapefile.Reader(shp=shp, dbf=dbf)

In [11]:
lats = []
lons = []
country = []
for shprec in sf.shapeRecords():
    name_long = unicode(shprec.record[18], 'cp1252').encode('utf8')
    country.append(name_long)
    lat, lon = map(list, zip(*shprec.shape.points))
    indices = shprec.shape.parts.tolist()
    lat = [lat[i:j] + [float('NaN')] for i, j in zip(indices, indices[1:]+[None])]
    lon = [lon[i:j] + [float('NaN')] for i, j in zip(indices, indices[1:]+[None])]
    lat = list(itertools.chain.from_iterable(lat))
    lon = list(itertools.chain.from_iterable(lon))
    lats.append(lat)
    lons.append(lon)

df = pd.DataFrame({'x': lats, 'y': lons, 'country': country})
cds = ColumnDataSource(df)
p = figure(width=800)
country_patches = p.patches('x', 'y', source=cds, line_color='white')
hover = HoverTool(renderers=[country_patches], tooltips=[('Country', '@country')])
p.add_tools(hover)

In [12]:
show(p)